Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [36]:
torcs_kill = 'killall torcs-bin'
torcs_cmd = 'torcs -nofuel -nodamage -t 500000 -r %s '
driver_cmd = 'java -jar torcs.jar'

race_xml = 'quickrace.xml'

trackTemplate = '<section name="%%IDX%%"><attstr name="name" val="%%TRACK_NAME%%"/><attstr name="category" val="%%TRACK_TYPE%%"/></section>'

tracks = [("dirt-1","dirt"),("dirt-2","dirt"),("dirt-3","dirt"),("dirt-4","dirt"),("dirt-5","dirt"),("dirt-6","dirt"),("mixed-1","dirt"),
("mixed-2","dirt"),("b-speedway","oval"),("c-speedway","oval"),("d-speedway","oval"),("e-speedway","oval"),("e-track-5","oval"),("f-speedway", "oval"),("g-speedway","oval"),("michigan","oval"),("aalborg","road"),
("brondehach","road"),("g-track-1","road"),("ole-road-1","road"),("e-track-4","road"),("street-1","road"),("alpine-1","road"),
("corkscrew","road"),("e-track-2","road"),("e-track-6","road"),("g-track-2","road"),
("ruudskogen","road"),("wheel-1","road"),("alpine-2","road"),("eroad","road"),
("e-track-3","road"),("forza","road"),("g-track-3","road"),("wheel-2","road"),("spring","road")]

In [45]:
def getTrackCmd(i, trackName, trackType):
    return trackTemplate.replace("%%TRACK_NAME%%", trackName).replace("%%TRACK_TYPE%%", trackType).replace('%%IDX%%', str(i+1))

def generateXML():
    path = "./track.xml.tmpl"
    
    tracks_xml = ''
    picks = np.random.choice(range(len(tracks)), 5, replace=False)
    for i,t in enumerate(picks):
        tracks_xml += getTrackCmd(i, tracks[t][0], tracks[t][1]) + '\n'
    
    
    with open(path, "r") as f:
        content = f.read()
    f.close()

    content = content.replace('%%TRACK%%', tracks_xml)
                            
    with open(race_xml, "w") as f:
        f.write(content)
    f.close()

In [46]:
generateXML()

In [5]:
def start_torcs():
    Popen(["torcs", "-d -nofuel -nodamage -t 100000", "-r", os.path.join(os.getcwd(),race_xml)])

In [7]:
def start_evaluation():
        
    generateXML()

    start_torcs()

    results = []
    port, proc = (3002, Popen([driver_cmd], stdout=PIPE, stderr=PIPE))
    notComplete = True
    while driver_proc:
        retcode = proc.poll()
        if retcode is not None:
            notComplete = False
        else:
            time.sleep(.1)

        if retcode != 0 and retcode is not None:
            #raise RuntimeError(proc.stderr.read())
            print(proc.stderr.read())
        if retcode == 0:    
            results.append(proc.stdout.read())

    usefulres = []
    for res in results:
        if 'laptime' in res:
            print res
            usefulres.append(res)
                
    return usefulres

In [ ]:
# Runs driver and torcs and reads from cmd